#### Shoulder Press Tracker

[My Final Project in depth explanation here](https://gavin-thomas.github.io/Shoulder-Press.html)

#### #1 Install and Import Necessary Libraries



In [ ]:
# First I installed mediapipe, which is a general "detection" pre-built deep-learning (computer vision) library library
pip install mediapipe


# Next I installed opencv, which is a computer vision library that allows us to use our webcam.
pip install opencv-python

In [2]:
import cv2
import mediapipe as mp
# Numpy for Trig :)
import numpy as np

# A mediapipe solution is like a model that we are grabbing
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [7]:
# Here is our VIDEO FEED

# Setup video capture device
cap = cv2.VideoCapture(0)

# This loops through our video feed 
while cap.isOpened():
    # cap.read is like us getting the current feed from webcam
    # frame variable gives us the image from our webcam, ret is return var
    ret,frame = cap.read()
    #pipe through mediapipe feed, and image from webcam (frame)
    cv2.imshow('Mediapipe Feed',frame)

    # check if we try to break out of screen, this function helps us break out of the while loop
    # 0xFF helps us figure out which key we are hitting on our keyboard
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# if we break out of our video feed we will release our video feed and destroy windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

#### #2 Make Detections

In [8]:
# Here is our VIDEO FEED

# Setup video capture device
cap = cv2.VideoCapture(0)

# Setup our detection confidence and our tracking confidence
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # Loop through our video feed 
    while cap.isOpened():
        # cap.read is like us getting the current feed from webcam
        # frame variable gives us the image from our webcam, ret is return var
        ret,frame = cap.read()

        # Let's try and make some detections now...
        # First we recolour our image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # This line makes the detection (we set up our pose model before)
        # We store detections inside results
        results = pose.process(image)

        # Write up image then colour the image again back to normal image format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Let's Render our image detection now
        # No need to draw point, by point, just use media pipe drawing utilities
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                # Dot colour and specifications
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                # Line colour and specifications
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        #pipe through mediapipe feed, and image from webcam (frame)
        cv2.imshow('Mediapipe Feed',image)

        # check if we try to break out of screen, this function helps us break out of the while loop
        # 0xFF helps us figure out which key we are hitting on our keyboard
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# if we break out of our video feed we will release our video feed and destroy windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

#### #3 Determining Joints

In [9]:
# Here is our VIDEO FEED

# Setup video capture device
cap = cv2.VideoCapture(0)

# Setup our detection confidence and our tracking confidence
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # Loop through our video feed 
    while cap.isOpened():
        # cap.read is like us getting the current feed from webcam
        # frame variable gives us the image from our webcam, ret is return var
        ret,frame = cap.read()

        # Let's try and make some detections now...
        # First we recolour our image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # This line makes the detection (we set up our pose model before)
        # We store detections inside results
        results = pose.process(image)

        # Write up image then colour the image again back to normal image format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # This block of code allows for the extraction of landmarks
        # if we don't have detections or get an error we just pass through
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Let's Render our image detection now
        # No need to draw point, by point, just use media pipe drawing utilities
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                # Dot colour and specifications
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                # Line colour and specifications
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        #pipe through mediapipe feed, and image from webcam (frame)
        cv2.imshow('Mediapipe Feed',image)

        # check if we try to break out of screen, this function helps us break out of the while loop
        # 0xFF helps us figure out which key we are hitting on our keyboard
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# if we break out of our video feed we will release our video feed and destroy windows
cap.release()
cv2.destroyAllWindows()

In [7]:
# How many landmarks??
len(landmarks)

33

In [5]:
#print each landmark from particular landmark map
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [9]:
# Access the left shoulder landmark as loop and display it from array
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7799567
y: 0.84146
z: -0.41408777
visibility: 0.99849945

In [10]:
# What number is the LEFT shoulder in our array of landmarks?
mp_pose.PoseLandmark.LEFT_SHOULDER

<PoseLandmark.LEFT_SHOULDER: 11>

#### #4 Calculating Angles

In [4]:
# Let's define a function to calculate the angle between the left hip, left shoulder, and left elbow
def calculate_angle(a,b,c):
    a = np.array(a) # First Joint
    b = np.array(b) # Middle Joint
    c = np.array(c) # End Joint

    # Now lets do some Trigonometry!
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    # Our hinge joints cannot go more than 360 degrees hence this equation
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [36]:
# Get Left Shoulder x and y values
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

In [34]:
hip, shoulder, elbow

([0.7262412309646606, 0.8725166916847229],
 [0.8130804300308228, 1.3414616584777832],
 [0.8314734697341919, 1.7969979047775269])

In [37]:
calculate_angle(hip, shoulder, elbow)

15.541339769588362

In [ ]:
# Let's just quickly find out what my width and height of video feed is!
# Initialize video capture device
cap = cv2.VideoCapture(0)

# Get width and height of video feed
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# Print width and height of video feed
print(f"Video feed width: {width}, height: {height}")

# Release video capture device
cap.release()

In [10]:
# Here is our VIDEO FEED

# Setup video capture device
cap = cv2.VideoCapture(0)

# Setup our detection confidence and our tracking confidence
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # Loop through our video feed 
    while cap.isOpened():
        # cap.read is like us getting the current feed from webcam
        # frame variable gives us the image from our webcam, ret is return var
        ret, frame = cap.read()

        # Let's try and make some detections now...
        # First we recolour our image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # This line makes the detection (we set up our pose model before)
        # We store detections inside results
        results = pose.process(image)

        # Write up image then colour the image again back to normal image format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # This block of code allows for the extraction of landmarks
        # if we don't have detections or get an error we just pass through

        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get the X and Y values of the hip, shoulder, and elbow
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, shoulder, elbow)
            
            # Now let's visualize our angle data (pass through image from webcam and angle)
            # Use array multiplication (grab elbow coordinate and multiply by my webcam dimensions)
            # We do the multiplication to get the proper image dimensions (then we convert to a tuple which cv2 expects)
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        # Let's Render our image detection now
        # No need to draw point, by point, just use media pipe drawing utilities
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                # Dot colour and specifications
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                # Line colour and specifications
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        #pipe through mediapipe feed, and image from webcam (frame)
        cv2.imshow('Mediapipe Feed',image)

        # check if we try to break out of screen, this function helps us break out of the while loop
        # 0xFF helps us figure out which key we are hitting on our keyboard
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# if we break out of our video feed we will release our video feed and destroy windows
cap.release()
cv2.destroyAllWindows()

#### #5 Dumbell Shoulder Press Tracker

In [11]:

# Count Shoulder Press Reps
counter = 0
# Stage represents the up or down phase of the curl
stage = None



# Here is our VIDEO FEED
# Setup video capture device
cap = cv2.VideoCapture(0)

# Setup our detection confidence and our tracking confidence
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # Loop through our video feed 
    while cap.isOpened():
        # cap.read is like us getting the current feed from webcam
        # frame variable gives us the image from our webcam, ret is return var
        ret, frame = cap.read()

        # Let's try and make some detections now...
        # First we recolour our image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # This line makes the detection (we set up our pose model before)
        # We store detections inside results
        results = pose.process(image)

        # Write up image then colour the image again back to normal image format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # This block of code allows for the extraction of landmarks
        # if we don't have detections or get an error we just pass through

        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get the X and Y values of the hip, shoulder, and elbow
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, shoulder, elbow).round(3)
            
            # Now let's visualize our angle data (pass through image from webcam and angle)
            # Use array multiplication (grab elbow coordinate and multiply by my webcam dimensions)
            # We do the multiplication to get the proper image dimensions (then we convert to a tuple which cv2 expects)
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Now for some logical if statements for the shoulder press counter
            if angle > 160:
                stage = 'up'
            if angle < 80 and stage == 'up':
                stage = 'down'
                counter += 1
                print(counter)
                       
        except:
            pass
        
        # Now let's render the shoulder press ticker
        # To do this let's first setup a status box (last line ='-1' fills box with colour)
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Now for our REP data
        cv2.putText(image,'Rep',(15,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,0), 2, cv2.LINE_AA)            
        
        # Now for our STAGE data
        cv2.putText(image,'Stage',(65,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,0), 2, cv2.LINE_AA) 

        # Let's Render our image detection now
        # No need to draw point, by point, just use media pipe drawing utilities
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                # Dot colour and specifications
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                # Line colour and specifications
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        #pipe through mediapipe feed, and image from webcam (frame)
        cv2.imshow('Mediapipe Feed',image)

        # check if we try to break out of screen, this function helps us break out of the while loop
        # 0xFF helps us figure out which key we are hitting on our keyboard
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# if we break out of our video feed we will release our video feed and destroy windows
cap.release()
cv2.destroyAllWindows()